In [1]:
# Uncomment this to install dependencies for this notebook
# !pip install gym
# !pip install tqdm
# !pip install tensorboard
# !pip install matplotlib
# !pip install JSAnimation

In [2]:
# uncomment this to enable jax gpu preallocation, might lead to memory issues

import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

In [3]:
from typing import Sequence, List, Tuple, Optional

import numpy as np
import jax
import jax.numpy as jnp
import flax.linen as nn

In [4]:
from hypernn.jax.embedding_module import FlaxEmbeddingModule
from hypernn.jax.weight_generator import FlaxWeightGenerator
from hypernn.jax.hypernet import FlaxHyperNetwork
from hypernn.jax.utils import count_jax_params


In [5]:
from typing import Optional, Any, Dict

class CustomFlaxEmbeddingModule(FlaxEmbeddingModule):
    def setup(self):
        self.embedding = nn.Embed(self.num_embeddings, self.embedding_dim)

    def __call__(self, inp: Optional[Any] = None):
        indices = jnp.arange(0, self.num_embeddings)
        return {"embedding":self.embedding(indices)}


In [6]:
class CustomFlaxWeightGenerator(FlaxWeightGenerator):
    def setup(self):
        self.dense1 = nn.Dense(32)
        self.dense2 = nn.Dense(self.hidden_dim, use_bias=False)

    def __call__(self, embedding_module_output: Dict[str, jnp.array], inp: Optional[Any] = None):
        x = self.dense1(embedding_module_output["embedding"])
        x = nn.relu(x)
        x = self.dense2(x)
        return {"params":x.reshape(-1)}

In [7]:
class MLP(nn.Module):

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(256, use_bias=True)(x)
        x = nn.tanh(x)
        x = nn.Dense(256, use_bias=True)(x)
        x = nn.tanh(x)
        x = nn.Dense(4, use_bias=False)(x)
        return x

In [8]:
from tensorflow_probability.substrates import jax as tfp

import jax
import functools

@functools.partial(jax.jit, static_argnames=('apply_fn', 'method'))
def generate_params(apply_fn, hypernetwork_params, method):
    generated_params, embedding_module_output, weight_generator_output = apply_fn({'params':hypernetwork_params}, method=method)
    return generated_params

@functools.partial(jax.jit, static_argnames=('apply_fn'))
def sample_actions(apply_fn, hypernetwork_params, obs, generated_params):
    out =  apply_fn({'params':hypernetwork_params}, inp=[jnp.expand_dims(jnp.array(obs), 0)], generated_params=generated_params, has_aux=False)
    # out = jnp.squeeze(out)
    # dist = tfp.distributions.Categorical(logits=out, allow_nan_stats=False)
    # seed = np.random.randint(0, 100000)
    # rng = jax.random.PRNGKey(seed)
    # action = dist.sample(seed=rng)
    # return action
    return out

def rollout(env, apply_fn, generate_params_fn, hypernetwork_params, render=False, seed: int = 0) -> float:
    obs = env.reset()
    done = False
    observations, actions, rewards, rendereds = [], [], [], []
    generated_params = generate_params(apply_fn, hypernetwork_params, generate_params_fn)
    while not done:
        rendered = None
        if render:
            rendered = env.render(mode="rgb_array")
            rendereds.append(rendered)

        out = sample_actions(apply_fn, hypernetwork_params, obs, generated_params)
        out = jnp.squeeze(out)
        dist = tfp.distributions.Categorical(logits=out)
        rng = jax.random.PRNGKey(np.random.randint(0, 10**10))
        action = dist.sample(seed=rng).item()

        next_obs, r, done, _ = env.step(action)

        observations.append(obs)
        actions.append(action)
        rewards.append(r)

        obs = next_obs

    num_steps = len(observations)
    env.close()
    return observations, actions, rewards, rendereds, num_steps

In [9]:
import optax
from flax.training import train_state  # Useful dataclass to keep train state

def create_train_state(rng, hypernetwork, learning_rate, input_shape):
    """Creates initial `TrainState`."""
    params = hypernetwork.init(rng, [jnp.ones(input_shape)])['params']
    # Exponential decay of the learning rate.
    scheduler = optax.exponential_decay(
        init_value=learning_rate, 
        transition_steps=1000,
        decay_rate=0.99)

    tx = optax.chain(
        optax.clip_by_global_norm(10.0),
        # optax.scale_by_adam(),  # Use the updates from adam.
        # optax.scale_by_schedule(scheduler),  # Use the learning rate from the scheduler.
        # # Scale updates by -1 since optax.apply_updates is additive and we want to descend on the loss.
        # optax.scale(-1.0)
        optax.adam(learning_rate)
    )
    return train_state.TrainState.create(
        apply_fn=hypernetwork.apply, params=params, tx=tx)

In [10]:
@functools.partial(jax.jit, static_argnames=('apply_fn'))
def train_step(apply_fn, state, observations, actions, discounted_rewards):
    def loss_fn(params):
        logits = apply_fn({'params':params}, inp=[observations], has_aux=False)
        dist = tfp.distributions.Categorical(logits=jnp.squeeze(logits))
        log_probs = jnp.squeeze(dist.log_prob(actions))
        loss = jnp.sum(discounted_rewards * log_probs)
        return -1*loss
    grad_fn = jax.value_and_grad(loss_fn)
    loss, grads = grad_fn(state.params)
    return state.apply_gradients(grads=grads), loss, grads


In [11]:
import os
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

def get_tensorboard_logger(
    experiment_name: str, base_log_path: str = "tensorboard_logs"
):
    log_path = "{}/{}_{}".format(base_log_path, experiment_name, datetime.now())
    train_writer = SummaryWriter(log_path, flush_secs=10)
    full_log_path = os.path.join(os.getcwd(), log_path)
    print(
        "Follow tensorboard logs with: python -m tensorboard.main --logdir '{}'".format(full_log_path)
    )
    return train_writer


In [12]:
import pandas as pd
import tqdm

def flatten(d):
    df = pd.json_normalize(d, sep='_')
    return df.to_dict(orient='records')[0]

def discount_reward(rews, gamma: float = 0.99):
    n = len(rews)
    rtgs = np.zeros_like(rews)
    for i in reversed(range(n)):
        rtgs[i] = rews[i] + gamma*(rtgs[i + 1] if i + 1 < n else 0)
    return rtgs

def reinforce(
        num_epochs,
        env,
        hypernetwork,
        seed: int = 0,
        lr: float = 0.0001,
        gamma: float = 0.99,
    ):
    writer = get_tensorboard_logger("HypernetworkJaxRL")

    rng = jax.random.PRNGKey(seed)
    rng, init_rng = jax.random.split(rng)
    state = create_train_state(init_rng, hypernetwork, lr, (1,8))

    bar = tqdm.tqdm(np.arange(num_epochs))

    try:
        for i in bar:
            observations, actions, rewards, _, num_steps = rollout(env, hypernetwork.apply, hypernetwork.generate_params, state.params, seed=seed)
            discounted_rewards = discount_reward(np.array(rewards), 0.99)
            discounted_rewards = discounted_rewards - np.mean(discounted_rewards)
            discounted_rewards = discounted_rewards / (
                np.std(discounted_rewards) + 1e-10
            )

            observations = jnp.array(observations)
            actions = jnp.array(actions)
            discounted_rewards = jnp.array(discounted_rewards)

            state, loss, grads = train_step(hypernetwork.apply, state, observations, actions, discounted_rewards)
            grad_dict = {k:dict(grads[k]) for k in grads.keys()}
            grad_dict = flatten(grad_dict)

            grad_dict = {k: {kk: np.sum(vv).item() for kk, vv in v.items()}
                        for k, v in grad_dict.items()}
            grad_dict = flatten(grad_dict)

            metrics = {"loss":loss.item(), "rewards":np.sum(rewards), "num_steps":num_steps, **grad_dict}

            for key in metrics:
                writer.add_scalar(key, metrics[key], i)

            bar.set_description('Loss: {}, Sum Reward: {}'.format(loss.item(), np.sum(rewards)))
        return hypernetwork, state
    except KeyboardInterrupt as e:
        return hypernetwork, state


In [13]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)


gpu


In [14]:
target_network = MLP()

count_jax_params(target_network, inputs=jnp.zeros((1,8)))

69120

In [15]:
hyper = FlaxHyperNetwork.from_target(
                target_network=target_network,
                target_input_shape=[(1,8)],
                embedding_module=CustomFlaxEmbeddingModule,
                weight_generator=CustomFlaxWeightGenerator,
                embedding_dim = 32,
                num_embeddings = 512
)
count_jax_params(hyper, inputs=[jnp.zeros((1,8))])

21760

In [16]:
hyper

FlaxHyperNetwork(
    # attributes
    target_network = MLP()
    target_input_shape = [(1, 8)]
    target_treedef = PyTreeDef(CustomNode(<class 'flax.core.frozen_dict.FrozenDict'>[()], [{'params': {'Dense_0': {'bias': *, 'kernel': *}, 'Dense_1': {'bias': *, 'kernel': *}, 'Dense_2': {'kernel': *}}}]))
    embedding_module = CustomFlaxEmbeddingModule
    weight_generator = CustomFlaxWeightGenerator
    embedding_dim = 32
    num_embeddings = 512
    hidden_dim = None
    num_target_parameters = 69120
    embedding_module_kwargs = {}
    weight_generator_kwargs = {}
    target_weight_shapes = [(256,), (8, 256), (256,), (256, 256), (256, 4)]
)

In [17]:
import gym

env = gym.make("LunarLander-v2")

In [18]:
hyper, state = reinforce(100000, env, hyper, seed=10, lr=0.0001)

Follow tensorboard logs with: python -m tensorboard.main --logdir '/home/shyam/Code/hyper-nn/notebooks/jax/tensorboard_logs/HypernetworkJaxRL_2022-04-07 14:27:52.947233'


Loss: -67.25050354003906, Sum Reward: 222.75297797490637:   2%|▏         | 1736/100000 [50:01<47:11:27,  1.73s/it]   


In [27]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib ipympl

plt.style.use('ggplot')
# Imports specifically so we can render outputs in Jupyter.
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display
from celluloid import Camera
from IPython.display import HTML

# def rollout(env, apply_fn, generate_params_fn, hypernetwork_params, render=False, seed: int = 0) -> float:

def render_rollout(model, state):
    fig = plt.figure("Animation",figsize=(7,5))
    camera = Camera(fig)
    ax = fig.add_subplot(111)
    observations, actions, rewards, rendereds, num_steps = rollout(gym.make("LunarLander-v2"), model.apply, model.generate_params, state.params, render=True)
    frames = []
    for r in rendereds:
        frame = ax.imshow(r)
        ax.axis('off')
        camera.snap()
        frames.append([frame])
    animation = camera.animate(blit=False, interval=50)
    # display(animations.to_html5_video())
    animation.save('lunar_lander.mp4')
    return animation



In [26]:
_ = render_rollout(hyper, state)

TypeError: setup() got an unexpected keyword argument 'clear_temp'